## 1
The constant is 873 dollars. You could interpret this as that families with no chilren and no income spend around 900 dollars on recreation. 

For every dollar of annual income, .0012 dollars are spent on recreation. 

If the annualincome2 is the squared term, you could think of that as saying that this rate increases by .00004 for every extra dollar in cincome.

Families with kids spend 224 dollars less on recreation than those without. 

In [1]:
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.diagnostic import het_breuschpagan
import numpy as np
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'


In [2]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
df = pd.read_sql_query('select * from weatherinszeged',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96453 entries, 0 to 96452
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   date                 96453 non-null  datetime64[ns, UTC]
 1   summary              96453 non-null  object             
 2   preciptype           96453 non-null  object             
 3   temperature          96453 non-null  float64            
 4   apparenttemperature  96453 non-null  float64            
 5   humidity             96453 non-null  float64            
 6   windspeed            96453 non-null  float64            
 7   windbearing          96453 non-null  float64            
 8   visibility           96453 non-null  float64            
 9   loudcover            96453 non-null  float64            
 10  pressure             96453 non-null  float64            
 11  dailysummary         96453 non-null  object             
 12  tempdiff          

In [12]:
df['tempdiff']=df.apparenttemperature-df.temperature

In [13]:
X=df[['humidity','windspeed']]
y=df['tempdiff']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

In [16]:
X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

model = sm.OLS(y_train, X_train_const).fit()

In [17]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               tempdiff   R-squared:                       0.289
Model:                            OLS   Adj. R-squared:                  0.289
Method:                 Least Squares   F-statistic:                 1.567e+04
Date:                Tue, 01 Sep 2020   Prob (F-statistic):               0.00
Time:                        20:43:50   Log-Likelihood:            -1.3620e+05
No. Observations:               77162   AIC:                         2.724e+05
Df Residuals:                   77159   BIC:                         2.724e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4379      0.023    104.020      0.000       2.392       2.484
humidity      -3.0282      0.027   -113.311      0.000      -3.081      -2.976
windspeed     -0.1190      0.001   -157.870      0.000      -0.120      -0.117
==============================================================================
Omnibus:                     3096.421   Durbin-Watson:                   1.993
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3635.635
Skew:                          -0.471   Prob(JB):                         0.00
Kurtosis:                       3.492   Cond. No.                         88.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

A 100% increase in humidity is associated with a three degree decrease of apparent temperature as compared to temperature. An increase of windspeed of one unit leads to a .1 decreae of apparent temperature as compared to temperature. In the absence of either, you would expect the difference to be 2.4

They are all statistically significant. I would have expected the effect of humidity to have bee nteh opposite. Ie. it would have increased apparent temperature as compared to actual temperature. Perhaps this is caused by some unobserved factor.

### Next, include the interaction of humidity and windspeed to the model above and estimate the model using OLS. Are the coefficients statistically significant? Did the signs of the estimated coefficients for humidity and windspeed change? Interpret the estimated coefficients.


In [20]:
X['wind_humidity']=X.humidity*X.windspeed
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)
X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

model = sm.OLS(y_train, X_train_const).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               tempdiff   R-squared:                       0.342
Model:                            OLS   Adj. R-squared:                  0.342
Method:                 Least Squares   F-statistic:                 1.335e+04
Date:                Tue, 01 Sep 2020   Prob (F-statistic):               0.00
Time:                        20:54:01   Log-Likelihood:            -1.3322e+05
No. Observations:               77162   AIC:                         2.664e+05
Df Residuals:                   77158   BIC:                         2.665e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.1056      0.037      2.837      0.005       0.033       0.179
humidity          0.1488      0.048      3.111      0.002       0.055       0.243
windspeed         0.0889      0.003     32.463      0.000       0.083       0.094
wind_humidity    -0.2943      0.004    -78.736      0.000      -0.302      -0.287
==============================================================================
Omnibus:                     3874.274   Durbin-Watson:                   1.993
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7621.547
Skew:                          -0.369   Prob(JB):                         0.00
Kurtosis:                       4.351   Cond. No.                         193.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

How to interpret the interaction of wind and humidity? As wind increases, (apparent temp - temp) decreases, as well as the other way around. Now humidity and windspeed has a positive effect on perceived temp, too. 

### Load the houseprices data from Thinkful's database.
Run your house prices model again and interpret the results. Which features are statistically significant, and which are not?


In [24]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
df = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()
drop_cols=[]
for col in df.columns:
    if df[col].isna().mean()>.4:
        drop_cols=drop_cols+[col]
df_clean=df.drop(columns=drop_cols).dropna()
df_clean['secondflrexists']=0
df_clean.loc[df['secondflrsf']>0,'secondflrexists']=1 #can prob drop 2nd fl sq ft
df_clean['totalsf']=df_clean['firstflrsf']=+df_clean['secondflrsf']
df_clean['yrsbltqared']=df_clean['yearbuilt']*df_clean['yearbuilt']
y=df_clean['saleprice']
X=df_clean.drop(columns=['saleprice','id'])
X.mszoning.dtype==int
cat_cols=[c for c in X.columns if X[c].dtype=='O']
numeric_cols=[c for c in X.columns if X[c].dtype!='O']
niceneighborhoods=['NridgHt', 'NoRidge', 'Somerst', 'Timber', 'Veenker', 'StoneBr']
df_clean['nicehood']=0
df_clean.loc[df_clean.neighborhood.isin(niceneighborhoods),'nicehood']=1
X2 = df_clean[['overallqual','grlivarea', 'garagearea', 'totalbsmtsf','yearremodadd','yearbuilt','yrsbltqared','lotarea','nicehood']]
Y2 = df_clean.saleprice
X2 = sm.add_constant(X2)
results = sm.OLS(Y2, X2).fit()
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:              saleprice   R-squared:                       0.785
Model:                            OLS   Adj. R-squared:                  0.783
Method:                 Least Squares   F-statistic:                     440.4
Date:                Tue, 01 Sep 2020   Prob (F-statistic):               0.00
Time:                        21:13:48   Log-Likelihood:                -13104.
No. Observations:                1094   AIC:                         2.623e+04
Df Residuals:                    1084   BIC:                         2.628e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         -2.15e+07   5.77e+06     -3.723   

There are no statistically insignificant features. Overall quality, living area, garage area, total basement sf, year remodled, year built, lot area, and living in a nice neighborhood all had positive effects on the price. Year built squared had a negative one, meaning that as the year that it was built increases, it's marginal efffect decreases. 

You can't really say what has the 'most effect' because you're mostly comparing apples to oranges. Year that the house was built seems to have a larger effect than the year it was remodled Extra units in living area seem to have a larger effect than more basement square feet or the garage. Living in one of the nicer neighborhoods seems to have a large effect, though this might just be because I picked the neighborhoods that had higher saverage sales prices. All of this makes sense to me.